In [10]:
import numpy as np

In [12]:
def taxi_id_number(num_taxis):
    arr = np.arange(num_taxis)
    np.random.shuffle(arr)
    for i in range(num_taxis):
        yield arr[i]

In [13]:
ids = taxi_id_number(10)
print(next(ids))
print(next(ids))
print(next(ids))

2
6
3


In [4]:
def shift_info():
    start_times_and_freqs = [(0, 8), (8, 30), (16, 15)]
    indices = np.arange(len(start_times_and_freqs))
    while True:
        idx = np.random.choice(indices, p = [0.25, 0.5, 0.25])
        start = start_times_and_freqs[idx]
        yield (start[0], start[0] + 7.5, start[1])

In [14]:
def taxi_process(taxi_id_generator, shift_info_generator):
    taxi_id = next(taxi_id_generator)
    shift_start, shift_end, shift_mean_trips = next(shift_info_generator)
    actual_trips = round(np.random.normal(loc = shift_mean_trips, scale = 2))
    average_trip_time = 6.5 / shift_mean_trips * 60
    
    ##평균 운행 시간을 분 단위로 변환한다
    between_events_time = 1.0 / (shift_mean_trips - 1) * 60
    ##이 도시는 매우 효율적이라 모든 택시가 거의 항상 사용된다
    time = shift_start
    yield TimePoint(taxi_id, 'start_shift', time)
    deltaT = np.random.poisson(between_events_time) / 60
    time += deltaT
    for i in range(actual_trips):
        yield TimePoint(taxi_id, 'pick up', time)
        deltaT = np.random.poisson(average_trip_time) / 60
        time += deltaT
        yield TimePoint(taxi_id, 'drop off', time)
        deltaT = np.random.poisson(between_events_time) / 60
        time += deltaT
    deltaT = np.random.poisson(between_events_time) / 60
    time += deltaT
    yield TimePoint(taxi_id, 'end shift ', time)

In [15]:
from dataclasses import dataclass

@dataclass
class TimePoint:
    taxi_id : int
    name : str
    time : float
    
    def __lt__(self, other):
        return self.time < other.time

In [16]:
import queue

class Simulator:
    def __init__(self, num_taxis):
        self._time_points = queue.PriorityQueue()
        taxi_id_generator = taxi_id_number(num_taxis)
        shift_info_generator = shift_info()
        self._taxis = [taxi_process(taxi_id_generator, shift_info_generator) for i in range(num_taxis)]
        self._prepare_run()
        
    def _prepare_run(self):
        for t in self._taxis:
            while True:
                try:
                    e = next(t)
                    self._time_points.put(e)
                except:
                    break
                    
    def run(self):
        sim_time = 0
        while sim_time < 24:
            if self._time_points.empty():
                break
            p = self._time_points.get()
            sim_time = p.time
            print(p)

In [17]:
sim = Simulator(10)
sim.run()

TimePoint(taxi_id=6, name='start_shift', time=0)
TimePoint(taxi_id=6, name='pick up', time=0.1)
TimePoint(taxi_id=6, name='drop off', time=1.0)
TimePoint(taxi_id=6, name='pick up', time=1.15)
TimePoint(taxi_id=6, name='drop off', time=2.083333333333333)
TimePoint(taxi_id=6, name='pick up', time=2.15)
TimePoint(taxi_id=6, name='drop off', time=2.933333333333333)
TimePoint(taxi_id=6, name='pick up', time=3.0166666666666666)
TimePoint(taxi_id=6, name='drop off', time=3.85)
TimePoint(taxi_id=6, name='pick up', time=4.0)
TimePoint(taxi_id=6, name='drop off', time=4.85)
TimePoint(taxi_id=6, name='pick up', time=5.033333333333333)
TimePoint(taxi_id=6, name='drop off', time=5.933333333333334)
TimePoint(taxi_id=6, name='pick up', time=6.183333333333334)
TimePoint(taxi_id=6, name='drop off', time=7.033333333333333)
TimePoint(taxi_id=6, name='pick up', time=7.216666666666667)
TimePoint(taxi_id=6, name='drop off', time=7.95)
TimePoint(taxi_id=0, name='start_shift', time=8)
TimePoint(taxi_id=5, nam

In [9]:
np.random.normal(loc=30, scale=2)

28.784417595333252